**Connect google drive**

In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Wed Aug 12 08:31:43 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.57       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
'Colab Notebooks'   yolov3


**1) Clone the Darknet**



In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 14240, done.
remote: Total 14240 (delta 0), reused 0 (delta 0), pack-reused 14240
Receiving objects: 100% (14240/14240), 12.79 MiB | 1.99 MiB/s, done.
Resolving deltas: 100% (9677/9677), done.


**2) Compile Darknet using Nvidia GPU**


In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet


**3) Configure Darknet network for training YOLO V3**

#### 訓練前下面的Cell內容要做修改 (cfg設定檔)
- 這一行指令是要將yolov3_training.cfg裡面的classes參數，從80改成5(5是範例中的類別數)
- 我們使用sed來進行字串的取代，sed是UNIX內建的取代工具，可以將前面的物件取代成後面的
- ex: classes=80@classes=5@ 這一行指令會把classes=80取代成classes=5

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=5@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=5@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=5@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=30@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=30@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=30@' cfg/yolov3_training.cfg

#### 訓練前下面的Cell內容要做修改 (obj設定檔 * 2)
- classes從80改成5(範例中的類別數)
- obj.names裡面配置的是各個類別的名稱

In [ ]:
!echo -e "Tug\nFerry\nSailboat\nCruise\nUnknown" > data/obj.names
!echo -e 'classes= 5\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data


#### 下面的Cell執行後會開始下載預訓練的YOLO權重，會花一些時間

In [ ]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2020-07-30 07:47:32--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74.1’

darknet53.conv.74.1   2%[                    ]   4.33M   170KB/s    eta 9m 23s ^C


**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [ ]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
replace data/obj/images/httpcsvkmeuaeccjpg.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
!cp data/obj/images/httpcsvkmeuaeccjpg.jpg /content/darknet/test.jpg

In [ ]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/images/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

0 0.523305 0.092357 0.182203 0.140127

['0', '0.523305', '0.092357', '0.182203', '0.140127']
0 data/obj/images/httpssmediacacheakpinimgcomxdbbdbbbececacdecdcdfjpg.txt
0 0.523305 0.092357 0.182203 0.140127
0 0.684322 0.255892 0.258475 0.296296

['0', '0.684322', '0.255892', '0.258475', '0.296296']
1 data/obj/images/httpssmediacacheakpinimgcomxdfdfadcfeabjpg.txt
0 0.684322 0.255892 0.258475 0.296296
0 0.415000 0.309406 0.230000 0.361386

['0', '0.415000', '0.309406', '0.230000', '0.361386']
2 data/obj/images/httpftqncomymusicLxZeltonjohnjpg.txt
0 0.415000 0.309406 0.230000 0.361386
0 0.466463 0.370000 0.689024 0.668000

['0', '0.466463', '0.370000', '0.689024', '0.668000']
3 data/obj/images/httpimagesfandangocomrImageRendererredesignstaticimgnoxportraitjpgpcpcpcimagesmasterrepositoryperformerimagespjpg.txt
0 0.466463 0.370000 0.689024 0.668000
0 0.495652 0.385294 0.956522 0.747059

['0', '0.495652', '0.385294', '0.956522', '0.747059']
4 data/obj/images/httpcsvkmeuaeccjpg.txt
0 0.495652 0

In [ ]:
import glob
images_list = glob.glob("data/obj/images/*.jpg")
print(images_list)

['data/obj/images/httpgraphicsnytimescomimagessectionmoviesfilmographyWireImagejpg.jpg', 'data/obj/images/httpcsvkmeuaeccjpg.jpg', 'data/obj/images/httpftqncomymusicLxZeltonjohnjpg.jpg', 'data/obj/images/httpimagescontactmusiccomnewsimagesjerryseinfeldjpg.jpg', 'data/obj/images/httpimagesfandangocomrImageRendererredesignstaticimgnoxportraitjpgpcpcpcimagesmasterrepositoryperformerimagespjpg.jpg', 'data/obj/images/httpimagesmtvcomurimgidfiledocrootvhcomsitewideflipbooksimgdailyjpgenlargefalsemattetruematteColorblackquality.jpg', 'data/obj/images/httpiamediaimdbcomimagesMMVBMTAxNDUMzUwOTdeQTJeQWpwZBbWUMDUOTAyNTIVUXCRALjpg.jpg', 'data/obj/images/httpssmediacacheakpinimgcomxdbbdbbbececacdecdcdfjpg.jpg', 'data/obj/images/httpssmediacacheakpinimgcomxdfdfadcfeabjpg.jpg', 'data/obj/images/httpssmediacacheakpinimgcomxedaedabcbefbcbabbjpg.jpg', 'data/obj/images/httpikinjaimgcomgawkermediaimageuploadsWmIuhdsrcedidjpgjpg.jpg']


In [ ]:
train_list = images_list[:-2]
test_list = images_list[-2:]


In [ ]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(train_list)) 
file.close() 

#Create test.txt file
file = open("data/test.txt", "w") 
file.write("\n".join(test_list)) 
file.close() 


**6) Start the training**

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

 CUDA-version: 10010 (10010), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
yolov3_training
 0 : compute_capability = 750, cudnn_half = 0, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 4, batch = 64, time_steps = 1, train = 1 
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   3 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   4 Shortcut Layer: 1,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   5 conv    128       3 x 3/ 2    208 x 208 x  64 ->  104 x 104 x 128 1.595 BF
   6 conv     64       1 x 1/ 1    104 x 104 x 128 ->  104 x 104 x  64 0.177 BF
   7 ^C


In [ ]:
%cd /content/darknet

/content/darknet


In [ ]:
!mkdir -p /content/gdrive/My\ Drive/yolov3/inference_config_file2

In [ ]:
!cp -f /content/darknet/data/obj.names /content/gdrive/My\ Drive/yolov3/inference_config_file
!cp -f /content/darknet/data/obj.data /content/gdrive/My\ Drive/yolov3/inference_config_file
!cp -f /content/gdrive/My\ Drive/yolov3/yolov3_training_last.weights /content/gdrive/My\ Drive/yolov3/inference_config_file
!cp -f /content/darknet/cfg/yolov3_training.cfg /content/gdrive/My\ Drive/yolov3/inference_config_file


In [ ]:
!./darknet detector test ./data/obj.data ./cfg/yolov3_training.cfg /content/gdrive/My\ Drive/yolov3/yolov3_training_last.weights ./test.jpg

 CUDA-version: 10010 (10010), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
 0 : compute_capability = 750, cudnn_half = 0, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 1, batch = 16, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   3 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   4 Shortcut Layer: 1,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   5 conv    128       3 x 3/ 2    208 x 208 x  64 ->  104 x 104 x 128 1.595 BF
   6 conv     64       1 x 1/ 1    104 x 104 x 128 ->  104 x 104 x  64 0.177 BF
   7 conv    128       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x 128 1.595 BF
   8 Shortcut Layer: 5,  wt = 0, wn = 0, outputs: 104 x 104 x 128 0.001 

In [ ]:
!cp -f /content/darknet/cfg/yolov3_training.cfg ./yolov3_training.cfg 
!cp -f /content/gdrive/My\ Drive/yolov3/yolov3_training_last.weights ./yolov3_training_last.weights

In [ ]:
!python convert.py /content/darknet/cfg/yolov3_training.cfg /content/gdrive/My\ Drive/yolov3/yolov3_training_1000.weights ./yolov3.h5

2020-08-12 03:52:35.048509: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Loading weights.
Weights Header:  0 2 5 [64000]
Parsing Darknet config.
Creating Keras model.
Parsing section net_0
Parsing section convolutional_0
conv2d bn leaky (3, 3, 3, 32)
2020-08-12 03:52:37.541217: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-08-12 03:52:37.599249: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-08-12 03:52:37.600041: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla K80 computeCapability: 3.7
coreClock: 0.8235GHz coreCount: 13 deviceMemorySize: 11.17GiB deviceMemoryBandwidth: 223.96GiB/s
2020-08-12 03:52:37.600092: I tensorfl

In [ ]:
!python convert.py -h 

2020-08-12 03:47:03.655055: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
usage: convert.py [-h] [-p] [-w] config_path weights_path output_path

Darknet To Keras Converter.

positional arguments:
  config_path         Path to Darknet cfg file.
  weights_path        Path to Darknet weights file.
  output_path         Path to output Keras model file.

optional arguments:
  -h, --help          show this help message and exit
  -p, --plot_model    Plot generated Keras model and save as image.
  -w, --weights_only  Save as Keras weights file instead of model file.
